In [1]:
import RPi.GPIO as gpio
import time
import random
from time import sleep
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
import cv2
import threading
from ObjectDetector import loadModel , ROI , DetectLoc , DetectLoc_2 , DetectLoc_3 #, loadModel_Tf 

# from CNN_ModelMaker import Sequential , Dense , Flatten , ReLU , Softmax , Dropout
# from ConvProg_2 import Conv2D , MaxPool2D , Sequential , Dense , Flatten , ReLU , Softmax
from Modules import Sequential , Dense , Flatten , Dropout , ReLU , Softmax , TanH
import numpy as np
import os
import traceback

gpio.setmode(gpio.BOARD)
gpio.setwarnings(False)

In [2]:
a = time.localtime()
fileName = '/home/pi/Desktop/Models/model_50_4_2.h5'
# fileName = 'ConvModel.h5'
VideoName = '/home/pi/Desktop/Videos/Car_Recordings/' + f"REC_{fileName.split('/')[-1]}_{a.tm_mday}_{a.tm_mon}_{a.tm_year}-{a.tm_hour}:{a.tm_min}:{a.tm_sec}.avi"

In [3]:
traffic_sign = ''
__speed__ = 0
Labels = [#'Speed limit (30kmph)', 'Speed limit (60kmph)',
          'Stop', 'Turn right ahead', 'Turn left ahead' ,
          'Negative']
lower_red = np.array([0,50,50])
upper_red = np.array([10,255,255])

lower_red2 = np.array([170,50,50])
upper_red2 = np.array([180,255,255])

k_size = 3

kernel = np.ones((k_size,k_size),np.float32) / k_size ** 2

In [4]:
def FindLabel(frame,model) :
    img1 = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(img1,lower_red,upper_red)
    img1 = cv2.bitwise_and(img1,img1,mask=mask)
    img1 = cv2.filter2D(img1,-1,kernel)
    contours , hier = cv2.findContours(mask,
                                       cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) :
        for cnt in contours :
            x,y,w,h = cv2.boundingRect(cnt)
            if w > 30 and h > 30 :
                im = frame[y:y+h,x:x+w]
                im = cv2.resize(im,(model.input_shape[0],model.input_shape[1]))
                pred = model.predict(np.expand_dims(im/255.,axis=0)).ravel() * 100
                if np.max(pred) >= 95 :
                    pred_ = np.argmax(pred)
                    return [Labels[pred_],x,y,x+w,y+h]
    return None

In [5]:
cam = cv2.VideoCapture(0)

class CamView :
    
    def __init__(self,speed=True,TimeStamp=True,fps=True,saveVideo=True) :
        self.TimeStamp = TimeStamp
        self.speed = speed
        self.fps = fps
        self.saveVideo = saveVideo
        self.size = 400
        if self.saveVideo :
            self.result = cv2.VideoWriter(VideoName,
                         cv2.VideoWriter_fourcc('M','J','P','G'),
                         5, (self.size,self.size))
    
    def show(self,size=400,Model=None,color=(100,150,150)) :
        pTime = 0 
        t = threading.currentThread()
        try :
            while getattr(t, "do_run", True) :
                if self.fps :
                    cTime = time.time()
                    fps = 1/(cTime-pTime)
                    pTime = cTime
                ret , frame = cam.read()
                frame = cv2.resize(frame,(size,size))
                frame = cv2.flip(frame,-1)
#                 point = DetectLoc(frame,
#                                   pos=[10,100,120,200],
#                                   kernels=[50],#,100],
#                                   stride=10,
#                                   ratios=[(1,1),(1,1.25),(1.25,1),(1.25,1.25)],
#                                   model=Model,
#                                   best=99)
#                 point = DetectLoc_2(frame,
#                                     model=Model,
#                                     pos=[0,100,125,200],best=20)
#                 point = DetectLoc_3(frame,model=Model,best=90)
                point = FindLabel(frame,Model)
                #print(point)
                global traffic_sign 
                if point is not None :
                    self.detLabel = point[0]
                    traffic_sign = point[0]
                    frame = cv2.rectangle(frame,
                                          (point[1],point[2]),
                                          (point[3],point[4]),
                                          (255,255,255),
                                          2)
                    frame = cv2.putText(frame,
                                        traffic_sign,
                                        (point[3]+5,point[4]+5),
                                        cv2.FONT_HERSHEY_SIMPLEX,.45,
                                        (255,255,255),
                                        1,cv2.LINE_AA)
                else :
                    traffic_sign = ''
                if self.speed :
                    frame = cv2.putText(frame,
                                        "Speed : " + str(__speed__)+' KMPH',
                                        (10,330),
                                        cv2.FONT_HERSHEY_SIMPLEX,.5,
                                        color,
                                        1,cv2.LINE_AA)
                if self.TimeStamp :
                    frame = cv2.putText(frame,
                                        "TimeStamp : " + f'{time.localtime().tm_hour}:{time.localtime().tm_min}:{time.localtime().tm_sec}',
                                        (10,360),
                                        cv2.FONT_HERSHEY_SIMPLEX,.5,
                                        color,
                                        1,cv2.LINE_AA)
                if self.fps :
                    frame = cv2.putText(frame,"FPS : "+str(int(fps)),(10,390),
                                        cv2.FONT_HERSHEY_SIMPLEX,.5,color,
                                        1,cv2.LINE_AA)
                if self.saveVideo :
                    frame = cv2.putText(frame,"[REC]",(350,390),
                                        cv2.FONT_HERSHEY_SIMPLEX,.5,(0,0,255),
                                        1,cv2.LINE_AA)
                    self.result.write(frame)
                cv2.imshow('Car View',frame)
                cv2.waitKey(1)
        except Exception as e:
            cv2.destroyAllWindows()
            cam.release()
            self.result.release()
            traceback.print_exc()

In [6]:
class Motor() :
    def __init__ (self,Mot,err=None) :
        
        self.Mot = Mot
        
        for M in Mot :
            for key in M :
                if key != 'P' :
                    gpio.setup(M[key],gpio.OUT)
                    
        self.pwm_F_L = gpio.PWM(Mot[0]['E'],100)
        self.pwm_F_R = gpio.PWM(Mot[1]['E'],100)
        self.pwm_B_L = gpio.PWM(Mot[2]['E'],100)
        self.pwm_B_R = gpio.PWM(Mot[3]['E'],100)
        
        self.pwm_F_L.start(0)
        self.pwm_F_R.start(0)
        self.pwm_B_L.start(0)
        self.pwm_B_R.start(0)
        
        self.__speed__ = 0
        self.__error__ = err
        
        self.__maxSpeed__ = 100
        self.__maxTurn__ = 80
        
    def move(self,speed=0,turn=0,t=.5,lSpeed=None,rSpeed=None) :
        k = self.__maxSpeed__
        tu = self.__maxTurn__
        speed *= k
        turn *= tu
        if ( lSpeed is None ) and ( rSpeed is None ) :
            lSpeed = speed - turn
            rSpeed = speed + turn

            if lSpeed>k : lSpeed = k
            elif lSpeed<-k : lSpeed = -k
            if rSpeed>k : rSpeed = k
            elif rSpeed<-k : rSpeed = -k
        else :
            lSpeed = int(self.__maxSpeed__ * lSpeed / 100)
            rSpeed = int(self.__maxSpeed__ * rSpeed / 100)
            
#         global __speed__
#         __speed__ = max(lSpeed,rSpeed)

        #print(lSpeed,rSpeed)
        
        self.pwm_F_L.ChangeDutyCycle(abs(lSpeed))
        self.pwm_F_R.ChangeDutyCycle(abs(rSpeed))
        self.pwm_B_L.ChangeDutyCycle(abs(lSpeed))
        self.pwm_B_R.ChangeDutyCycle(abs(rSpeed))
        
        if lSpeed>0 :
            gpio.output(self.Mot[0]['I'],gpio.HIGH)
            gpio.output(self.Mot[0]['O'],gpio.LOW)
            gpio.output(self.Mot[2]['I'],gpio.HIGH)
            gpio.output(self.Mot[2]['O'],gpio.LOW)
        else :
            gpio.output(self.Mot[0]['O'],gpio.HIGH)
            gpio.output(self.Mot[0]['I'],gpio.LOW)
            gpio.output(self.Mot[2]['O'],gpio.HIGH)
            gpio.output(self.Mot[2]['I'],gpio.LOW)
        if rSpeed>0 :
            gpio.output(self.Mot[1]['I'],gpio.HIGH)
            gpio.output(self.Mot[1]['O'],gpio.LOW)
            gpio.output(self.Mot[3]['I'],gpio.HIGH)
            gpio.output(self.Mot[3]['O'],gpio.LOW)
        else :
            gpio.output(self.Mot[1]['O'],gpio.HIGH)
            gpio.output(self.Mot[1]['I'],gpio.LOW)
            gpio.output(self.Mot[3]['O'],gpio.HIGH)
            gpio.output(self.Mot[3]['I'],gpio.LOW)
        
        sleep(t)
        
    def stop(self,speed=0,t=1) :
        self.pwm_F_L.ChangeDutyCycle(speed)
        self.pwm_F_R.ChangeDutyCycle(speed)
        self.pwm_B_L.ChangeDutyCycle(speed)
        self.pwm_B_R.ChangeDutyCycle(speed)
        self.__speed__ = 0
        sleep(t)
        return

In [7]:
Labels = [#'Speed limit (30kmph)' ,
          #'Speed limit (60kmph)' ,
          'Stop',
          'Turn right ahead',
          'Turn left ahead',
          'Negative']

In [8]:
def moveCar() :
    t = threading.currentThread()
    while getattr(t, "do_run", True) :
        Label = traffic_sign
        global __speed__
        if Label.lower() == 'Speed limit (30kmph)'.lower() :
            __speed__ = 30
            sleep(.1)
            while True :
                car.move(t=1,lSpeed=15,rSpeed=46)
        elif Label.lower() == 'Speed limit (60kmph)'.lower() :
            __speed__ = 60
            sleep(.1)
            while True :
                car.move(t=1,lSpeed=20,rSpeed=60)
        elif Label.lower() == 'Turn left ahead'.lower() :
            car.move(lSpeed=13,rSpeed=42,t=1.5)
            t = 0
            while t < 7 :
                car.move(lSpeed=-80,rSpeed=100,t=.5)
                car.move(lSpeed=9,rSpeed=29,t=.5)
                t += 1
        elif Label.lower() == 'Turn right ahead'.lower() :
            car.move(lSpeed=13,rSpeed=42,t=1.5)
            t = 0
            while t < 7 :
                car.move(rSpeed=-80,lSpeed=100,t=.5)
                car.move(lSpeed=9,rSpeed=29,t=.5)
                t += 1
        elif Label.lower() == 'Stop'.lower() :
            car.move(0,0)
            __speed__ = 0
            sleep(1)
        else :
            car.move(t=.5,lSpeed=10,rSpeed=32)

In [9]:
camera = CamView(speed=False,TimeStamp=False)

In [10]:
Mot = [ {'P' : 'F_L' , 'I' : 26 , 'O' : 24 , 'E' : 22 },
        {'P' : 'F_R' , 'I' : 40 , 'O' : 38 , 'E' : 36 },
        {'P' : 'B_L' , 'I' : 35 , 'O' : 37 , 'E' : 33 },
        {'P' : 'B_R' , 'I' : 11 , 'O' : 13 , 'E' : 15 }
      ]
car = Motor(Mot)

In [11]:
# model = loadModel_Tf(fileName)
# model.summary()
model = loadModel(fileName)
model.Summary()

____________________________________________________________
Model Summary
____________________________________________________________
Layers                Input Shape           Output Shape         
____________________________________________________________
Flatten               (50, 50, 3)           7500
____________________________________________________________
Dense                 7500                  4
____________________________________________________________


In [12]:
camera_thread = threading.Thread(target=camera.show,args=(400,model,(255,255,255)))
camera_thread.start()
car_thread = threading.Thread(target=moveCar)
car_thread.start()

In [13]:
exit_ = input()
camera_thread.do_run = False
car_thread.do_run = False
car.stop()
camera.result.release()
cv2.destroyAllWindows()
cam.release()

e


In [14]:
gpio.cleanup()

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-74904af0ac3a>", line 35, in moveCar
    car.move(t=.5,lSpeed=10,rSpeed=32)
  File "<ipython-input-6-63ccffe00b29>", line 55, in move
    gpio.output(self.Mot[0]['I'],gpio.HIGH)
RuntimeError: Please set pin numbering mode using GPIO.setmode(GPIO.BOARD) or GPIO.setmode(GPIO.BCM)



### Straight - > speed = .4 turn = .24 t = _